---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices.

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [2]:
import pandas as pd

doc = []
with open('dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.Series(doc)
df.head(10)

0         03/25/93 Total time of visit (in minutes):\n
1                       6/18/85 Primary Care Doctor:\n
2    sshe plans to move as of 7/8/71 In-Home Servic...
3                7 on 9/27/75 Audit C Score Current:\n
4    2/6/96 sleep studyPain Treatment Pain Level (N...
5                    .Per 7/06/79 Movement D/O note:\n
6    4, 5/18/78 Patient's thoughts about current su...
7    10/24/89 CPT Code: 90801 - Psychiatric Diagnos...
8                         3/7/86 SOS-10 Total Score:\n
9             (4/10/71)Score-1Audit C Score Current:\n
dtype: object

In [318]:
def date_sorter():
    import numpy as np
    
    # Your code here
    # mm/dd/yy mm/dd/yyyy
    #s1_1 = df.str.extractall(r'(\d{1,2})[/-](\d{1,2})[/-](\d{2})')
    #s1_2 = df.str.extractall(r'(\d{1,2})[/-](\d{1,2})[/-](\d{4})')
    s1_1 = df.str.extractall(r'(\d{1,2})[/-](\d{1,2})[/-](\d{2})\b')
    s1_2 = df.str.extractall(r'(\d{1,2})[/-](\d{1,2})[/-](\d{4})\b')
    s1 = pd.concat([s1_1,s1_2])
    s1.reset_index(inplace=True)
    s1_index = s1['level_0']
    
    #
    s2 = df.str.extractall(r'((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[-.]* )((?:\d{1,2}[?:, -]*)\d{4})')
    s2.reset_index(inplace=True)
    s2_index = s2['level_0']
    
    s3 = df.str.extractall(r'((?:\d{1,2} ))?((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[?:, -]* )(\d{4})')
    s3.reset_index(inplace=True)
    s3_index = s3['level_0']
    
    s4 = df.str.extractall(r'(\d{1,2})[/](\d{4})')
    s4.reset_index(inplace=True)
    s4_index = s4['level_0']
    
    #drop s4 dates which is in s1
    s4 = s4.drop(s4[s4['level_0'].isin(s1_index)].index)
    
    s5_1= df.str.extractall(r'[a-z]?[^0-9](\d{4})[^0-9]')
    s5_2 = df.str.extractall(r'^(\d{4})[^0-9]')
    s5 = pd.concat([s5_1,s5_2])
    s5.reset_index(inplace=True)

    s5_index = s5['level_0']
    s5 = s5.drop(s5[s5['level_0'].isin(pd.concat([s2_index,s3_index,s4_index]))].index)
    #s5 = s5.drop(s5[s5['level_0'].isin(s3_index)].index)
    #s5 = s5.drop(s5[s5['level_0'].isin(s4_index)].index)
    
    a7_1= df.str.extractall(r'[a-z]?[^0-9](\d{4})[^0-9]')
    a7_2 = df.str.extractall(r'^(\d{4})[^0-9]')
    a7 = pd.concat([a7_1,a7_2])
    a7.reset_index(inplace=True)

    a7_index = a7['level_0']
    save=[]
    for i in a7_index:
        if not((i in s2_index.values) | (i in s3_index.values) | (i in s4_index.values)):
            save.append(i)
    save = np.asarray(save)
    a7 = a7[a7['level_0'].isin(save)]
    
    #s = s1.level_0.values.tolist()
    #s = np.asarray(s)
    s = pd.concat([s1_index,s2_index,s3_index,s4_index,s5_index])
    
    s1.columns=['level_0','match','month','day','year']
    s1['year']=s1['year'].apply(str)
    s1['year']=s1['year'].apply(lambda x: '19'+x if len(x)<=2 else x)
    
    s2[1] = s2[1].apply(lambda x: x.replace(',',''))
    s2['day'] = s2[1].apply(lambda x:x.split(' ')[0])
    s2['year'] = s2[1].apply(lambda x:x.split(' ')[1])
    s2.columns=['level_0','match','month','day-year','day','year']
    s2.drop('day-year',axis=1,inplace=True) 
    
    s3.columns=['level_0','match','day','month','year']
    s3['day'] = s3['day'].replace(np.nan,-99)
    s3['day'] = s3['day'].apply(lambda x: 1 if int(x)==-99 else x)

    s3['month'] = s3.month.apply(lambda x: x[:3])
    s3['month'] = pd.to_datetime(s3.month, format='%b').dt.month
    
    s4.columns=['level_0','match','month','year']
    s4['day']=1
  
    s5.columns=['level_0','match','year']
    s5['day']=1
    s5['month']=1
   
    final = pd.concat([s1,s2,s3,s4,s5])
    final['date'] =pd.to_datetime(final['month'].apply(str)+'/'+final['day'].apply(str)+'/'+final['year'].apply(str))
    final = final.sort_values(by='level_0').set_index('level_0')
    
    rep = final['date']
    answer = pd.Series([i[0] for i in sorted(enumerate(rep), key=lambda x:x[1])],np.arange(500))
    
    return answer

date_sorter()


0        9
1       84
2        2
3       53
4       28
5      474
6      153
7       13
8      129
9       98
10     111
11     225
12      31
13     171
14     191
15     486
16     335
17     415
18      36
19     323
20     405
21     422
22     375
23     380
24     345
25      57
26     481
27     436
28     104
29     299
      ... 
470    220
471    208
472    243
473    139
474    320
475    383
476    244
477    286
478    480
479    431
480    279
481    198
482    381
483    463
484    366
485    255
486    439
487    401
488    475
489    257
490    152
491    235
492    464
493    253
494    231
495    427
496    141
497    186
498    161
499    413
dtype: int64